In [13]:
from cimloader.databases import ConnectionInterface, ConnectionParameters, Parameter, QueryResponse
from cimloader.databases.mysql import MySQLConnection

In [14]:
params = ConnectionParameters(host= "localhost", database="cimhub_2023", username="gridappsd", password="gridappsd1234",
                               cim_profile='cimhub_2023', namespace="http://iec.ch/TC57/CIM100#")
mysql = MySQLConnection(params)
mysql.connect()

Could not connect to database


In [15]:
mysql.create_database(database='cimhub_2023', overwrite=False)


ProgrammingError: 1044 (42000): Access denied for user 'gridappsd'@'%' to database 'cimhub_2023'

In [ ]:
mysql.configure()
# mysql.create_tables()

In [ ]:
mysql.connect()

In [ ]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [ ]:
from cimgraph.databases import Parameter, ConnectionParameters
from cimgraph.databases.blazegraph.blazegraph import BlazegraphConnection
from cimgraph.models import FeederModel
import json
import time
# Blazegraph connection for running outside the container
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile='rc4_2021')
bg = BlazegraphConnection(params)


#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
# feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
# feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
feeder_mrid = "EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node
## Create Distributed CIM Model:
feeder = cim.Feeder(mRID=feeder_mrid)
start = time.perf_counter()
network_bg = FeederModel(connection=bg, container=feeder, distributed=False)
build_time = time.perf_counter() - start
print("blazegraph init time:", build_time)


cim_class = cim.ACLineSegment
start = time.perf_counter()
network_bg.get_all_edges(cim_class)
build_time = time.perf_counter() - start
print("blazegraph query time for", len(network_bg.graph[cim_class]), "objects:", build_time)

In [ ]:
mysql.upload_from_cimgraph(network_bg)

In [ ]:
start = time.perf_counter()
myresult = mysql.execute("SELECT * FROM ACLineSegment WHERE _mRID in ('3D75EB6B-DBB9-4852-9F51-D20629FD1DC6')")


for x in myresult:
    print(x)
build_time = time.perf_counter() - start
print("mysql query time:", build_time)

In [ ]:
cim.ACLineSegment()

In [ ]:
start = time.perf_counter()
myresult = mysql.execute("""SELECT * FROM ACLineSegment WHERE _EquipmentContainer->'$."@id"' = 'EE71F6C9-56F0-4167-A14E-7F4C71F10EAA' """)


for x in myresult:
    print(x)
build_time = time.perf_counter() - start
print("mysql query time for", len(myresult), " rows in", build_time)

In [ ]:
print(int(time.time()), time.time())

In [ ]:
print(network_bg.get_edges_query(cim.ACLineSegment))

In [ ]:
all_classes = []
parsed_classes = []
for class_type in list(network_bg.graph.keys()):
    all_classes.append(class_type.__name__)

while set(parsed_classes) != set(all_classes):
    for class_name in all_classes:
        if class_name not in parsed_classes:
            print(class_name)
            class_type = eval(f"cim.{class_name}")
            network_bg.get_all_edges(class_type)
            parsed_classes.append(class_name)
            all_classes = []
            for class_type in list(network_bg.graph.keys()):
                all_classes.append(class_type.__name__)
